In [9]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
# JsonOutputParser 불러오기
# -> 모델 출력이 valid JSON일 것을 기대하고, 그 JSON을 파이썬 객체(dict 등)로 변환해줌

json_parser = JsonOutputParser()
# JsonOutputParser 인스턴스 생성

template = """
    오직 JSON 문자열만 출력을 해라. 코드블록 금지
    키 : {{"title" : str, "bullets" : list[str], "tone" : "neutral|postive|negative"}}
    {text}
"""
# 프롬프트 템플릿 문자열 정의
# - "오직 JSON" : 모델에게 코드블록(```...```)없이 순수한 JSON만 내놓으라고 강하게 지시
# - 키 설명 {{"title":...}} : 중괄호를 두 번 쓴 이유는 템플릿 엔진에 그대로 '{' '}'를 전달하기 위함(=문자 그대로의 중괄호로 JSON 스키마 보여주기)
# - {text} : 실제 분석할 입력 텍스트(나중에 포맷 시 여기에 채워짐)
# 참고: "postive"는 오타로 보입니다. 올바른 값은 "positive" 입니다.

prompt_template = ChatPromptTemplate.from_template(
    template = template
)
# ChatPromptTemplate 객체 생성
# -> 위의 템플릿 문자열을 대화형 프롬프트로 사용하도록 만듦

chain = prompt_template | model | json_parser
# LCEL 문법으로 체인 구성
# 1) prompt_template -> 2) model(LLM) -> 3) json_parser
# 체인 실행 시: 템플릿 생성 → 모델이 응답(문자열 JSON) → json_parser가 JSON으로 파싱

chain
# 만든 체인 객체 확인(실행 준비 완료)


ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n    오직 JSON 문자열만 출력을 해라. 코드블록 금지\n    키 : {{"title" : str, "bullets" : list[str], "tone" : "neutral|postive|negative"}}\n    {text}\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000209E6F47D90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000209E6F475D0>, root_client=<openai.OpenAI object at 0x00000209E6F17850>, root_async_client=<openai.AsyncOpenAI object at 0x00000209E6F479D0>, model_name='gpt-4.1-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))
| JsonOutputParser()

In [12]:
text = """
오늘 아침에 프로젝트 마감이 임박했는데도 불구하고 팀원들과의 긴밀한 협력 덕분에 무사히 작업을 완료할 수 있었습니다. 특히 김 대리님의 자료 분석 능력이 빛을 발했고, 모두가 어려운 상황 속에서도 긍정적인 태도를 잃지 않았습니다. 이 성과를 바탕으로 다음 프로젝트도 성공적으로 이끌어낼 수 있을 거라는 확신이 듭니다
"""
result = chain.invoke({"text" : text})

In [13]:
result

{'title': '프로젝트 마감 성공',
 'bullets': ['프로젝트 마감 임박 상황에서도 팀원들과 긴밀히 협력함',
  '김 대리님의 뛰어난 자료 분석 능력 발휘',
  '어려운 상황 속에서도 긍정적인 태도 유지',
  '이번 성과를 바탕으로 다음 프로젝트 성공 기대'],
 'tone': 'positive'}

In [14]:
print(type(result))

<class 'dict'>
